In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,Jan-99,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
charity_df= charity_df.drop(columns=["NAME","EIN", "AFFILIATION", "CLASSIFICATION"])
charity_df.head()
                                     

,APPLICATION_TYPE,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,ProductDev,Association,1,0,N,5000,1
1,T3,Preservation,Co-operative,1,Jan-99,N,108590,1
2,T5,ProductDev,Association,1,0,N,5000,0
3,T3,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,INCOME_AMT_Jan-99,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,INCOME_AMT_Jan-99,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Remove loan status target from features data
y = charity_df.IS_SUCCESSFUL
X = charity_df

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 1.000


In [13]:
X_train_scaled = np.asarray(X_train_scaled)
y_train = np.asarray(y_train)
X_test_scaled = np.asarray(X_test_scaled)
y_test= np.asarray(y_test)

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 61us/sample - loss: 0.1673 - accuracy: 0.9548
Epoch 2/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.0061 - accuracy: 0.9993
Epoch 3/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.0016 - accuracy: 0.9997
Epoch 4/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.0017 - accuracy: 0.9998
Epoch 5/50
25724/25724 [==============================] - 1s 33us/sample - loss: 0.0014 - accuracy: 0.9998
Epoch 6/50
25724/25724 [==============================] - 1s 34us/sample - loss: 8.8387e-04 - accuracy: 0.9999
Epoch 7/50
25724/25724 [==============================] - 1s 37us/sample - loss: 2.6779e-04 - accuracy: 0.9999
Epoch 8/50
25724/25724 [==============================] - 1s 37us/sample - loss: 0.0021 - accuracy: 0.9997
Epoch 9/50
25724/25724 [==============================] - 1s 34us/sample - loss: 3.1682e-04 - accuracy: 1.0000
Ep

In [15]:
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                984       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 1,297
Trainable params: 1,297
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
   32/25724 [..............................] - ETA: 6:07 - loss: 4.4892e-09 - accuracy: 1.0000
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1024/25724 [>.............................] - ETA: 13s - loss: 3.5138e-09 - accuracy: 1.0000 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2048/25724 [=>............................] - ETA: 7s - loss: 1.2233e-08 - accuracy: 1.0000 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 3072/25724 [==>...........................] - ETA: 4s - loss: 9.1547e-09 - accuracy: 1.0000
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4096/25724 [===>..........................] - ETA: 3s - loss: 7.5378e-09 - accuracy: 1.0000
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5152/25724 [=====>........................] - ETA: 3s - loss: 6.5431e-09 - accuracy: 1.0000
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 7136/25724 [=======>......................] - ETA: 2s - los

 2848/25724 [==>...........................] - ETA: 1s - loss: 3.5181e-09 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 3872/25724 [===>..........................] - ETA: 1s - loss: 3.5138e-09 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 4928/25724 [====>.........................] - ETA: 1s - loss: 8.5865e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5952/25724 [=====>........................] - ETA: 1s - loss: 7.1150e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 7008/25724 [=======>......................] - ETA: 0s - loss: 6.0482e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 8096/25724 [========>.....................] - ETA: 0s - loss: 5.2400e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 9184/25724 [=========>....................] - ETA: 0s - loss: 4.6231e-07 - accuracy: 1.0000
Epoch 0

 6016/25724 [======>.......................] - ETA: 1s - loss: 3.2844e-09 - accuracy: 1.0000
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 6976/25724 [=======>......................] - ETA: 0s - loss: 3.3360e-09 - accuracy: 1.0000
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 8032/25724 [========>.....................] - ETA: 0s - loss: 3.3237e-09 - accuracy: 1.0000
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 9024/25724 [=========>....................] - ETA: 0s - loss: 6.1228e-09 - accuracy: 1.0000
Epoch 00005: saving model to checkpoints/weights.05.hdf5
10048/25724 [==========>...................] - ETA: 0s - loss: 5.8527e-09 - accuracy: 1.0000
Epoch 00005: saving model to checkpoints/weights.05.hdf5
11104/25724 [===========>..................] - ETA: 0s - loss: 5.6242e-09 - accuracy: 1.0000
Epoch 00005: saving model to checkpoints/weights.05.hdf5
12032/25724 [=============>................] - ETA: 0s - loss: 5.4772e-09 - accuracy: 1.0000
Epoch 0

 9536/25724 [==========>...................] - ETA: 0s - loss: 1.0340e-07 - accuracy: 1.0000
Epoch 00007: saving model to checkpoints/weights.07.hdf5
10624/25724 [===========>..................] - ETA: 0s - loss: 9.3125e-08 - accuracy: 1.0000
Epoch 00007: saving model to checkpoints/weights.07.hdf5
11648/25724 [============>.................] - ETA: 0s - loss: 8.5212e-08 - accuracy: 1.0000
Epoch 00007: saving model to checkpoints/weights.07.hdf5
12704/25724 [=============>................] - ETA: 0s - loss: 7.8373e-08 - accuracy: 1.0000
Epoch 00007: saving model to checkpoints/weights.07.hdf5
13696/25724 [==============>...............] - ETA: 0s - loss: 7.2901e-08 - accuracy: 1.0000
Epoch 00007: saving model to checkpoints/weights.07.hdf5
14784/25724 [================>.............] - ETA: 0s - loss: 6.7766e-08 - accuracy: 1.0000
Epoch 00007: saving model to checkpoints/weights.07.hdf5
15840/25724 [=================>............] - ETA: 0s - loss: 6.3463e-08 - accuracy: 1.0000
Epoch 0

13312/25724 [==============>...............] - ETA: 0s - loss: 3.7520e-08 - accuracy: 1.0000
Epoch 00009: saving model to checkpoints/weights.09.hdf5
14368/25724 [===============>..............] - ETA: 0s - loss: 3.4968e-08 - accuracy: 1.0000
Epoch 00009: saving model to checkpoints/weights.09.hdf5
15424/25724 [================>.............] - ETA: 0s - loss: 5.8074e-08 - accuracy: 1.0000
Epoch 00009: saving model to checkpoints/weights.09.hdf5
16416/25724 [==================>...........] - ETA: 0s - loss: 5.4745e-08 - accuracy: 1.0000
Epoch 00009: saving model to checkpoints/weights.09.hdf5
17440/25724 [===================>..........] - ETA: 0s - loss: 5.1721e-08 - accuracy: 1.0000
Epoch 00009: saving model to checkpoints/weights.09.hdf5
18464/25724 [====================>.........] - ETA: 0s - loss: 4.9003e-08 - accuracy: 1.0000
Epoch 00009: saving model to checkpoints/weights.09.hdf5
19488/25724 [=====================>........] - ETA: 0s - loss: 4.6587e-08 - accuracy: 1.0000
Epoch 0

...

 8896/25724 [=========>....................] - ETA: 0s - loss: 4.5208e-11 - accuracy: 1.0000
Epoch 00092: saving model to checkpoints/weights.92.hdf5
 9888/25724 [==========>...................] - ETA: 0s - loss: 4.2629e-11 - accuracy: 1.0000
Epoch 00092: saving model to checkpoints/weights.92.hdf5
10880/25724 [===========>..................] - ETA: 0s - loss: 4.1366e-11 - accuracy: 1.0000
Epoch 00092: saving model to checkpoints/weights.92.hdf5
11904/25724 [============>.................] - ETA: 0s - loss: 3.9508e-11 - accuracy: 1.0000
Epoch 00092: saving model to checkpoints/weights.92.hdf5
12928/25724 [==============>...............] - ETA: 0s - loss: 3.8513e-11 - accuracy: 1.0000
Epoch 00092: saving model to checkpoints/weights.92.hdf5
13952/25724 [===============>..............] - ETA: 0s - loss: 4.0416e-11 - accuracy: 1.0000
Epoch 00092: saving model to checkpoints/weights.92.hdf5
14976/25724 [================>.............] - ETA: 0s - loss: 4.2169e-11 - accuracy: 1.0000
Epoch 0

12032/25724 [=============>................] - ETA: 0s - loss: 2.0214e-10 - accuracy: 1.0000
Epoch 00094: saving model to checkpoints/weights.94.hdf5
12992/25724 [==============>...............] - ETA: 0s - loss: 1.9166e-10 - accuracy: 1.0000
Epoch 00094: saving model to checkpoints/weights.94.hdf5
13952/25724 [===============>..............] - ETA: 0s - loss: 1.8150e-10 - accuracy: 1.0000
Epoch 00094: saving model to checkpoints/weights.94.hdf5
14912/25724 [================>.............] - ETA: 0s - loss: 1.7094e-10 - accuracy: 1.0000
Epoch 00094: saving model to checkpoints/weights.94.hdf5
15936/25724 [=================>............] - ETA: 0s - loss: 1.6175e-10 - accuracy: 1.0000
Epoch 00094: saving model to checkpoints/weights.94.hdf5
16960/25724 [==================>...........] - ETA: 0s - loss: 1.5774e-10 - accuracy: 1.0000
Epoch 00094: saving model to checkpoints/weights.94.hdf5
18016/25724 [====================>.........] - ETA: 0s - loss: 1.4964e-10 - accuracy: 1.0000
Epoch 0

15040/25724 [================>.............] - ETA: 0s - loss: 3.6717e-11 - accuracy: 1.0000
Epoch 00096: saving model to checkpoints/weights.96.hdf5
16032/25724 [=================>............] - ETA: 0s - loss: 3.6680e-11 - accuracy: 1.0000
Epoch 00096: saving model to checkpoints/weights.96.hdf5
16992/25724 [==================>...........] - ETA: 0s - loss: 3.5814e-11 - accuracy: 1.0000
Epoch 00096: saving model to checkpoints/weights.96.hdf5
17952/25724 [===================>..........] - ETA: 0s - loss: 3.5423e-11 - accuracy: 1.0000
Epoch 00096: saving model to checkpoints/weights.96.hdf5
18976/25724 [=====================>........] - ETA: 0s - loss: 9.8790e-11 - accuracy: 1.0000
Epoch 00096: saving model to checkpoints/weights.96.hdf5
20032/25724 [======================>.......] - ETA: 0s - loss: 9.5252e-11 - accuracy: 1.0000
Epoch 00096: saving model to checkpoints/weights.96.hdf5
20992/25724 [=======================>......] - ETA: 0s - loss: 9.2017e-11 - accuracy: 1.0000
Epoch 0

17280/25724 [===================>..........] - ETA: 0s - loss: 3.3093e-11 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
18304/25724 [====================>.........] - ETA: 0s - loss: 3.2807e-11 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
19328/25724 [=====================>........] - ETA: 0s - loss: 3.4125e-11 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
20352/25724 [======================>.......] - ETA: 0s - loss: 3.3521e-11 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
21376/25724 [=======================>......] - ETA: 0s - loss: 3.3533e-11 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
22400/25724 [=========================>....] - ETA: 0s - loss: 3.4860e-11 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
23424/25724 [==========================>...] - ETA: 0s - loss: 3.5819e-11 - accuracy: 1.0000
Epoch 0

20096/25724 [======================>.......] - ETA: 0s - loss: 4.6895e-11 - accuracy: 1.0000
Epoch 00100: saving model to checkpoints/weights.100.hdf5
21120/25724 [=======================>......] - ETA: 0s - loss: 4.7183e-11 - accuracy: 1.0000
Epoch 00100: saving model to checkpoints/weights.100.hdf5
22144/25724 [========================>.....] - ETA: 0s - loss: 4.6033e-11 - accuracy: 1.0000
Epoch 00100: saving model to checkpoints/weights.100.hdf5
23168/25724 [==========================>...] - ETA: 0s - loss: 4.5158e-11 - accuracy: 1.0000
Epoch 00100: saving model to checkpoints/weights.100.hdf5
24192/25724 [===========================>..] - ETA: 0s - loss: 4.5339e-11 - accuracy: 1.0000
Epoch 00100: saving model to checkpoints/weights.100.hdf5
25724/25724 [==============================] - 1s 51us/sample - loss: 4.4786e-11 - accuracy: 1.0000
8575/1 - 0s - loss: 0.0024 - accuracy: 0.9998
Loss: 0.004898743628505828, Accuracy: 0.9997667670249939


In [18]:
print(len(X_train))

25724


In [19]:
number_input_features = len(X_train)
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: Shapes (25724, 8) and (40, 24) are incompatible

In [15]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [16]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: Error when checking input: expected dense_3_input to have shape (27317,) but got array with shape (36,)